# Distributed

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lukeconibear/intro_ml/blob/main/docs/04_distributed.ipynb)

In [1]:
# if you're using colab, then install the required modules
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    pass

Distributing training over multiple devices generally uses either:

- Data parallelism
    - Single model copied to multiple devices.
    - Each device processes different batch of data.
- Model parallelism
    - Model split over multiple devices.
    - Each device processes a single batch of data together.
    
This lesson focuses on data parallelism.

## [TensorFlow](https://www.tensorflow.org/guide/distributed_training)

You can use `tf.distribute.Strategy` to distribute models and training over multiple machines with minimal code changes.

MirrorStrategy

All-reduce

Define a strategy.

Within the strategy scope, compile and fit then model.

global batch size

```python
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model.compile(...)
    model.fit(...)
```

### [Mirrored Strategy](https://www.tensorflow.org/guide/distributed_training#mirroredstrategy)

Supports distributed train

[`TF_CONFIG`](https://www.tensorflow.org/guide/distributed_training#setting_up_the_tf_config_environment_variable)

```python
strategy = tf.distribute.MirroredStrategy()


```

In [4]:
import tensorflow as tf

2022-03-21 11:53:32.632593: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-21 11:53:32.632608: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
def get_compiled_model():
    # Make a simple 2-layer densely-connected neural network.
    inputs = tf.keras.Input(shape=(784,))
    x = tf.keras.layers.Dense(256, activation="relu")(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    outputs = tf.keras.layers.Dense(10)(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [6]:
def get_dataset():
    batch_size = 32
    num_val_samples = 10000

    # Return the MNIST dataset in the form of a [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Preprocess the data (these are Numpy arrays)
    x_train = x_train.reshape(-1, 784).astype("float32") / 255
    x_test = x_test.reshape(-1, 784).astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    # Reserve num_val_samples samples for validation
    x_val = x_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    x_train = x_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size),
    )

In [7]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1


2022-03-21 11:53:56.173312: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-21 11:53:56.173334: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-21 11:53:56.173346: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (UOL-LAP-5G6CZH3): /proc/driver/nvidia/version does not exist
2022-03-21 11:53:56.173523: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
# # Open a strategy scope.
# with strategy.scope():
#     # Everything that creates variables should be under the strategy scope.
#     # In general this is only model construction & `compile()`.
#     model = get_compiled_model()

# # Train the model on all available devices.
# train_dataset, val_dataset, test_dataset = get_dataset()
# model.fit(train_dataset, epochs=2, validation_data=val_dataset)

# # Test the model on all available devices.
# model.evaluate(test_dataset)

## Fault tolerance

Use Model Checkpointing (saw in the previous lesson) to be able to recover a model from a previous epoch if the training fails.

In [16]:
import os

# Prepare a directory to store all the checkpoints.
checkpoint_dir = f"{os.getcwd()}/../models/checkpoints"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [17]:
def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return tf.keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model()

In [18]:
def run_training(epochs=1):
    # Create a MirroredStrategy.
    strategy = tf.distribute.MirroredStrategy()

    # Open a strategy scope and create/restore the model
    with strategy.scope():
        model = make_or_restore_model()

    callbacks = [
        # This callback saves a SavedModel every epoch
        # We include the current epoch in the folder name.
        tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_dir + "/checkpoint-{epoch}", 
            save_freq="epoch"
        )
    ]
    model.fit(
        train_dataset,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=val_dataset,
        verbose=2,
    )

In [22]:
# # Running the first time creates the model
# run_training(epochs=1)

In [23]:
# # Calling the same function again will resume from where we left off
# run_training(epochs=1)

### Dataset caching

Add data caching to an example

and

### Prefetch data

Examples of how to distribute deep learning on a High Performance Computer (HPC).

## Install Python environments

First, install the Python environments for the required HPC.

## ARC4

### Miniconda installer
```bash
# download miniconda (x86_64 for ARC4)
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh

# run miniconda, read terms, and set path
. Miniconda3-latest-Linux-x86_64.sh
```

### Conda environment

#### Clone pre-created environments

```bash
# clone - tensorflow 2.7.0 and ray
conda env create --file tf_ray_arc4.yml

# clone - pytorch 1.10 and ray
module load gnu/8.3.0
conda env create --file pytorch_ray_arc4.yml
```

#### Create your own

```bash
# create new - tensorflow 2.7.0 and ray
conda create -n tf_ray_arc4 -c conda-forge python==3.9.* cudatoolkit==11.2.* cudnn==8.1.*
conda activate tf_ray_arc4
pip install -U pip
pip install tensorflow==2.7.0
pip install -U ray
pip install -U ray[tune]

# create new - pytorch (1.10) and ray
module load gnu/8.3.0
conda create -n pytorch_ray_arc4 pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch
conda activate pytorch_ray_arc4
pip install -U ray
pip install -U ray[tune]

# create new - jax
conda create -n jax python=3.8 cudatoolkit=11.2 cudatoolkit-dev=11.2 cudnn=8.2
conda activate jax
pip install -U jax
pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html
```

## Bede

### Miniconda installer
```bash
# Replace <project> with your project code
export DIR=/nobackup/projects/<project>/$USER

# download miniconda (ppc64le for Bede's hardware, not x86_64 as for ARC4)
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-ppc64le.sh
 
# run miniconda
sh Miniconda3-latest-Linux-ppc64le.sh -b -p $DIR/miniconda
source miniconda/bin/activate
 
# update conda and set channels
conda update conda -y
conda config --prepend channels conda-forge
conda config --prepend channels https://public.dhe.ibm.com/ibmdl/export/pub/software/server/ibm-ai/conda/
conda config --prepend channels https://opence.mit.edu
```

This is what my `~/.condarc` ends up as:
```bash
channel_priority: flexible
channels:
  - https://opence.mit.edu
  - https://public.dhe.ibm.com/ibmdl/export/pub/software/server/ibm-ai/conda/
  - conda-forge
  - defaults
```

### Conda environment

#### Clone pre-created environments

```bash
# clone - tensorflow 2.7.0 and ray
conda env create --file tf_bede.yml

# clone - pytorch 1.10 and ray
conda env create --file pytorch_bede.yml

# clone - pytorch 1.9.0, cuda 10.2, and pytorch_geometric 2.0.3
module load gcc # require this for some of the libraries
conda env create --file pytorch_geometric_bede.yml
```

#### Create your own

```bash
# create an environment for pytorch
conda create -n pytorch pytorch torchvision cudatoolkit=10.2
 
# create an environment for tensorflow
conda create -n tf tensorflow

# create an environment for pytorch geometric
module load gcc
conda create -n pytorch_geometric pytorch cudatoolkit=10.2
conda activate pytorch_geometric

pip install torch-scatter
pip install torch-sparse
pip install torch-geometric
pip install torch-cluster
```

## JADE-2

### Miniconda installer

```bash
...
```

### Conda environment

#### Clone pre-created environments

```bash
...
```

#### Create your own

```bash
...
```

## Jupyter Notebook to HPC

It's preferable to use a static job on the HPC. To do this, you could test out different ideas locally in a Jupyter Notebook, then when ready convert this to an executable script (`.py`) and move it over. 

...

## Examples

These examples use [Ray Train](https://docs.ray.io/en/latest/train/train.html) in a static job on a HPC.
Ray handles most of the complexity of distributing the work, with minimal changes to your [TensorFlow](https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras) or [PyTorch](https://pytorch.org/tutorials/beginner/dist_overview.html) code.

- Python script examples:
  - TensorFlow
    - MNIST end-to-end: [`tensorflow_mnist_example.py`](https://github.com/lukeconibear/intro_ml/blob/main/distributed/tensorflow_mnist_example.py).  
    - MNIST tuning: [`tensorflow_tune_mnist_example.py`](https://github.com/lukeconibear/intro_ml/blob/main/distributed/tensorflow_tune_mnist_example.py).  
    - Train linear model with Ray Datasets: [`tensorflow_linear_dataset_example.py`](https://github.com/lukeconibear/intro_ml/blob/main/distributed/tensorflow_linear_dataset_example.py).  
  - PyTorch
    - Linear: [`pytorch_train_linear_example.py`](https://github.com/lukeconibear/intro_ml/blob/main/distributed/pytorch_train_linear_example.py).  
    - Fashion MNIST: [`pytorch_train_fashion_mnist_example.py`](https://github.com/lukeconibear/intro_ml/blob/main/distributed/pytorch_train_fashion_mnist_example.py).  
    - HuggingFace Transformer: [`pytorch_transformers_example.py`](https://github.com/lukeconibear/intro_ml/blob/main/distributed/pytorch_transformers_example.py).  
    - Tune linear model with Ray Datasets: [`pytorch_tune_linear_dataset_example.py`](https://github.com/lukeconibear/intro_ml/blob/main/distributed/pytorch_tune_linear_dataset_example.py).  
- Then submit the job to HPC (choose one and update the Python script within it):
  - [ARC4](https://arcdocs.leeds.ac.uk/systems/arc4.html) (SGE)  
    - CPU: [`ray_train_on_arc4_cpu.bash`](https://github.com/lukeconibear/intro_ml/blob/main/distributed/ray_train_on_arc4_cpu.bash).  
    - GPU: [`ray_train_on_arc4_gpu.bash`](https://github.com/lukeconibear/intro_ml/blob/main/distributed/ray_train_on_arc4_gpu.bash).  
  - [Bede](https://bede-documentation.readthedocs.io/en/latest/) (SLURM)
    - GPU: [`ray_train_on_bede.bash`](https://github.com/lukeconibear/intro_ml/blob/main/distributed/ray_train_on_bede.bash).  
  - [JADE-2](http://docs.jade.ac.uk/en/latest/index.html) (SLURM)
    - GPU: ...



https://keras.io/guides/distributed_training/

Synchronous data-parallel training on all available GPUs:

In [ ]:
# distribution_strategy = tf.distribute.MirrorStratergy()
# with distribution_strategy.scope():
#     # Everything that creates variables should be under the strategy scope.
#     # In general this is only model construction and compile()
#     model = build_model()
#     model.compile(optimiser, loss)
#     model.fit(dataset, epochs=epochs, callbacks=callbacks)  

should the `model.fit` call be inside or outside the scope?

#$ -cwd

not

#$ -cwd -V

so have to specific the reproducible environment with the job submission (not copied over from the terminal)

## Exercises

```{admonition} Exercise 1

...

```

## {ref}`Solutions <distributed>`

## Key Points

```{important}

- [x] _..._

```

## Further information

### Good practices

- Batch the dataset with the global batch size e.g., for 8 devices each capable of a btach of 64 use the global batch size of 512 (= 8 * 64).  
- ...

### Other options

- [Horovod](https://horovod.ai/)
- [DeepSpeed](https://www.deepspeed.ai/)
 
### Resources

- ...